In [1]:
from torch_geometric.loader import DataLoader
from torch_geometric import seed_everything
from torch_geometric.transforms import Distance, Cartesian
from imports.TrainModel import TrainModel
seed_everything(0)

from models.spline import nvs_no_skip as splinenet

model = splinenet().to('cuda')
model
!rm ../data/contact_extraction10/{train,test,val}/processed/*

tm = TrainModel('/home/hussain/tactile/data/contact_extraction10/', model, n_epochs=150, transform=Cartesian(cat=False), features='pol', augment=False)

/bin/bash: /home/hussain/miniconda3/envs/tactile/lib/libtinfo.so.6: no version information available (required by /bin/bash)


Processing...


{'train_prop': 0.6, 'augment': False, 'delta_t': 25000000.0, 'margin': -25000000.0, 'max_events_thresh': 2000, 'n': 644, 'kNN': 32, 'node_features': 'pol'}


Done!
Processing...


{'train_prop': 0.6, 'augment': False, 'delta_t': 25000000.0, 'margin': -25000000.0, 'max_events_thresh': 2000, 'n': 644, 'kNN': 32, 'node_features': 'pol'}


Done!
Processing...


{'train_prop': 0.6, 'augment': False, 'delta_t': 25000000.0, 'margin': -25000000.0, 'max_events_thresh': 2000, 'n': 644, 'kNN': 32, 'node_features': 'pol'}


Done!


#### tm.train()

In [7]:
min(tm.val_losses)

0.022077545086204103

In [4]:
tm.test()

0.011305167100377001

In [24]:
import time 
from tqdm.auto import tqdm
t = []
def validate(self):
    loss = 0
    for i, data in enumerate(tqdm(self.val_loader)):
        now = time.time()
        data = data.to('cuda')
        end_point = self.model(data)
        end = time.time() - now
        t.append(end)
        print(end)
        loss += self.loss_func(end_point[0], data.y).detach().item()
    loss /= len(self.val_data)
    return loss

validate(tm)

  0%|          | 0/33 [00:00<?, ?it/s]

0.14057564735412598
0.17262601852416992
0.1955254077911377
0.15732598304748535
0.12617993354797363
0.13708806037902832
0.1367342472076416
0.19402194023132324
0.1806349754333496
0.16005754470825195
0.13409996032714844
0.1878058910369873
0.19119000434875488
0.19316840171813965
0.1704239845275879
0.1675717830657959
0.20548200607299805
0.18075847625732422
0.14330005645751953
0.1307077407836914
0.17769408226013184
0.20035505294799805
0.18609166145324707
0.1927506923675537
0.1511704921722412
0.14408493041992188
0.11974000930786133
0.10378074645996094
0.12209033966064453
0.17664790153503418
0.1758272647857666
0.13147425651550293
0.2036294937133789


0.0227090794536652

In [26]:
import numpy as np
np.mean(t)

0.1633519692854448

In [23]:
tm.model.to('cuda')

nvs_no_skip(
  (conv1): SplineConv(1, 64, dim=3)
  (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): SplineConv(64, 128, dim=3)
  (bn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): SplineConv(128, 256, dim=3)
  (bn3): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): SplineConv(256, 512, dim=3)
  (bn4): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=32768, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=2, bias=True)
)

In [11]:
from imports.ExtractContactCases import ExtractContactCases

ex = ExtractContactCases('/home/hussain/tactile/data/contact_extraction10/', '/home/hussain/tactile/dataset_ENVTACT_new2.bag')
ex.extract()

parsing bag: 0it [00:00, ?it/s]

  0%|          | 0/645 [00:00<?, ?it/s]

saving
386 129 129
